In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-127088")
exp = Experiment(workspace=ws, name="udacity-AzureML1")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-127088
Azure region: southcentralus
Subscription id: e8f628b3-bb5b-4edf-947a-8637ca6ea7c2
Resource group: aml-quickstarts-127088


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

cpu_cluster_name = "cpu-cluster-1"
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
#ps = ### YOUR CODE HERE ###
ps = RandomParameterSampling(
    {
        '--C' : uniform(0, 1),
        '--max_iter' : choice(50, 100, 200)
    }
)

# Specify a Policy
#Your Code Here
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval = 1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
 ### YOUR CODE HERE ###
est = SKLearn(source_directory='./', 
                    entry_script='train.py', 
                    compute_target=compute_target)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(
    hyperparameter_sampling = ps, 
    primary_metric_name = 'Accuracy',
    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, 
    max_total_runs = 100,
    max_concurrent_runs = 4,
    policy = policy,
    estimator = est
)

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [22]:
import joblib
# Get your best run and save the model from that run.
### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
joblib.dump(value= best_run, filename = 'model.joblib')
#print(best_run.get_file_names())
#model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'outputs/best-trained-model.pkl')
#model = best_run.register_model(model_name="banking_udacity", model_path= model_path)



TypeError: can't pickle _thread.RLock objects

In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(url)


In [12]:
from train import clean_data
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from azureml.core.run import Run
from sklearn.model_selection import train_test_split
# Use the clean_data function to clean your data.
x, y = clean_data(ds)
dataset = x.join(y)
train_data, test_data =  train_test_split(dataset, random_state = 223)

In [20]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    enable_early_stopping = True,
    primary_metric= "AUC_weighted",
    training_data= train_data,
    label_column_name="y",
    enable_onnx_compatible_models=True,
    n_cross_validations= 3)


In [21]:
# Submit your automl run

### YOUR CODE HERE ###

remote_run = exp.submit(automl_config, show_output = False)
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

WARNING - Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [23]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

from azureml.automl.runtime.onnx_convert import OnnxConverter
best_run_model, onnx_model= remote_run.get_output(return_onnx_model=True)
OnnxConverter.save_onnx_model(onnx_model, file_path="./automl_model.onnx")



'\nfrom azureml.automl.runtime.onnx_convert import OnnxConverter\nbest_aml_marketing_model, onnx_model= automl_run.get_output(return_onnx_model=True)\nOnnxConverter.save_onnx_model(onnx_model, "./automl_model.onnx")\n\nbest_run, onnx_mdl = remote_run.get_output(return_onnx_model=True)\nfrom azureml.automl.runtime.onnx_convert import OnnxConverter\nonnx_fl_path = "./best_model.onnx"\nOnnxConverter.save_onnx_model(onnx_mdl, onnx_fl_path)\n\n'